In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.09275078773498535
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 43.55729768399424
200000 84.92233638153559
300000 104.0974337297238
400000 106.1191956798107
500000 106.14744689053472
600000 107.07211975242784
700000 109.4770484109015
800000 104.10674042395424
900000 103.5642765583363
1000000 102.71396834111303
1100000 102.94870691269742
1200000 102.79874294947257
1300000 84.48932652600547
1400000 41.317873515926095
1500000 102.57492681640618
1600000 102.39547765684073
1700000 102.06602974705761
1800000 100.645849097093
1900000 101.73748254485153
2000000 106.13634278888469
2100000 102.42349223653574
2200000 101.32904220928668
2300000 101.75612151400942
2400000 102.15307299051408
2500000 102.35760679973122
2600000 97.6705944752256
2700000 24.47970665637746
2800000 98.66103124908695
2900000 97.66656772098105
3000000 97.

29400000 107.44073609506238
29500000 107.46750840684295
29600000 103.8430115943805
29700000 106.35264071742883
29800000 106.05829511595105
29900000 104.75980861323391
30000000 104.04582917574142
30100000 101.36914178369122
30200000 91.29095779658158
30300000 37.263008519455106
30400000 99.34913569461186
30500000 102.92406617882959
30600000 101.70671580830307
30700000 107.4193127087578
30800000 102.34187711194765
30900000 102.56370383801772
31000000 106.05901536546405
31100000 110.35590926118913
31200000 104.8788357545334
31300000 100.11332656809378
31400000 51.13324881321496
31500000 111.06829053116796
31600000 111.69014236465564
31700000 110.9267620262899
31800000 51.166510640732525
31900000 94.88759729594149
32000000 109.68255103028288
32100000 110.12099502655407
32200000 110.27026704241712
32300000 106.55723403467034
32400000 104.76626617418005
32500000 106.42093593549637
32600000 104.704642951911
32700000 105.66140999061936
32800000 102.32671855606279
32900000 24.963360357883534
33

59100000 96.15537148724701
59200000 24.287688125104644
59300000 102.28837514088335
59400000 101.76456156734152
59500000 100.2140341936478
59600000 102.72031556126544
59700000 105.06839756386898
59800000 102.74076673699417
59900000 105.65583261878322
60000000 98.9358423594307
60100000 43.97286622663502
60200000 106.88139912761395
60300000 106.68802761150803
60400000 105.21883005718848
60500000 104.40887255806983
60600000 104.64768109188391
60700000 105.05635153290758
60800000 103.5941860435565
60900000 65.49723559935002
61000000 79.00986937634197
61100000 100.36079121504592
61200000 97.61875900420458
61300000 97.61574773168212
61400000 103.3177222618663
61500000 95.28117498790962
61600000 91.743996356834
61700000 91.39487532203518
61800000 33.37481482872871
61900000 87.27592565095294
62000000 92.6664421418023
62100000 93.05848214776091
62200000 97.71088497391034
62300000 95.74448369810129
62400000 97.66604919576828
62500000 100.76434739236008
62600000 103.739122159241
62700000 34.934784

89200000 99.95572015053038
89300000 106.1872472390254
89400000 108.62771953576912
89500000 107.5927725203534
89600000 104.16109115954359
89700000 104.62965364347293
89800000 105.32098348878492
89900000 101.0380175595367
90000000 76.51097458768677
90100000 56.039870168097366
90200000 93.71275997538466
90300000 92.74165981394394
90400000 92.41802354282251
90500000 95.23774183981317
90600000 93.80258464416791
90700000 40.15951588909044
90800000 79.40955545718562
90900000 95.12451389533842
91000000 93.66593093030681
91100000 93.34299745614516
91200000 93.87435228152374
91300000 93.07699232210135
91400000 34.713091673814915
91500000 103.35443501519174
91600000 99.72340399988849
91700000 103.78650908071386
91800000 102.75721965018874
91900000 104.01245326861027
92000000 110.02311630437046
92100000 103.2221011729763
92200000 101.09639277933668
92300000 99.83003676953324
92400000 98.72399008682179
92500000 98.65561063080527
92600000 98.51787762036376
92700000 30.750371028600497
92800000 91.428

118800000 99.01208933103094
118900000 89.36466421505311
119000000 89.63830319208012
119100000 80.95743147322602
119200000 39.97903896152762
119300000 86.12641515666203
119400000 91.02688026165336
119500000 88.42664688747823
119600000 46.41835132571981
119700000 69.98961247909148
119800000 88.22613643814194
119900000 88.21365662073042
120000000 89.13894170176536
120100000 33.74138916809746
120200000 91.38755456612073
120300000 90.12403932298707
120400000 89.53389620410783
120500000 94.82205328279488
120600000 94.02955162678423
120700000 99.2454035444742
120800000 95.87194338564721
120900000 99.87352523402896
121000000 97.73167408317944
121100000 99.30198310755468
121200000 101.5752676858362
121300000 101.38286936676677
121400000 95.5458094896321
121500000 94.1006688084788
121600000 90.57921684793993
121700000 90.18392165211367
121800000 44.734897864417235
121900000 74.72459693709604
122000000 88.4590566079886
122100000 89.56443462095424
122200000 66.87662659538479
122300000 47.251019453

...

982900000 7.857748589195611
983000000 4.000498399132938
983100000 5.300281346540603
983200000 7.615225665173238
983300000 8.010765832263129
983400000 6.919354192078452
983500000 6.258510737284017
983600000 5.6724131751410045
983700000 1.8547199988753929
983800000 5.870444893242182
983900000 6.1738120247510855
984000000 6.686832703324531
984100000 5.940128201446749
984200000 5.894775404558883
984300000 3.5210491030007565
984400000 2.952506808348304
984500000 5.934223494460479
984600000 5.940538418076457
984700000 6.561347052758306
984800000 5.827739796903951
984900000 5.3699249430615605
985000000 2.8886027146061894
985100000 4.487884558689842
985200000 5.830679674544549
985300000 6.37111770830525
985400000 6.126352388462321
985500000 5.9154412293863246
985600000 4.123248497713463
985700000 2.6280197714487064
985800000 5.7172334188676475
985900000 5.783993399200184
986000000 6.654120693466769
986100000 5.926824082251447
986200000 5.277586917097815
986300000 3.1236870003995234
986400000 4

1011300000 0.8717552505216506
1011400000 2.8600466035367846
1011500000 2.7178560854886125
1011600000 3.2748763214958365
1011700000 2.9333479866204937
1011800000 2.9445742799300225
1011900000 1.4928825018341025
1012000000 1.708159071301067
1012100000 2.8394173217164953
1012200000 2.896851011876191
1012300000 3.1217927606376668
1012400000 2.550513302620019
1012500000 2.2942003691753188
1012600000 0.8681040650748809
1012700000 2.1411168051073126
1012800000 2.6735796221318067
1012900000 2.810355900701809
1013000000 2.576630978135138
1013100000 2.4575810166643595
1013200000 1.4860368627724747
1013300000 1.0986291117132823
1013400000 2.4439788658666397
1013500000 2.4048025945631983
1013600000 2.766293257882285
1013700000 2.2872365296292005
1013800000 2.2287544921073916
1013900000 0.8479941465053479
1014000000 1.6875648972885282
1014100000 2.2257007075059954
1014200000 2.518202159666408
1014300000 2.442694274667723
1014400000 2.2098885905929526
1014500000 1.5959951231743288
1014600000 0.91124

1037200000 2.6101298764857352
1037300000 3.1739240727190023
1037400000 3.6562560957593426
1037500000 2.8929136380503957
1037600000 3.205275684115697
1037700000 3.5489796415532844
1037800000 2.842467463509037
1037900000 3.4235950171244744
1038000000 3.2477134461399593
1038100000 2.756927425009549
1038200000 3.233192170108732
1038300000 3.2013443007878286
1038400000 2.8751396215214386
1038500000 3.3122728890964916
1038600000 3.272721714176187
1038700000 2.7825149806019427
1038800000 3.2818459172976144
1038900000 3.39929734016204
1039000000 3.00574194698295
1039100000 2.9909503017280232
1039200000 3.7147201439824102
1039300000 3.0659891548401896
1039400000 2.570547445671081
1039500000 3.8579723343064445
1039600000 3.407345678744954
1039700000 2.3466225412551402
1039800000 3.51063444523465
1039900000 3.5487208862009347
1040000000 2.945239108742603
1040100000 3.749791562345326
1040200000 4.46347032358892
best so far: 0
type: [1, 3, 1, 2, 23] 138
cases of this type: 6716184
1040300000 2.6554

1058400000 1.6538541394665396
1058500000 2.286059675785132
1058600000 1.8993876951794841
1058700000 1.7407369037419465
1058800000 1.975387229085034
1058900000 1.9837170565149977
1059000000 1.7061697708903294
1059100000 1.8678410662350713
1059200000 2.3174912437472064
1059300000 1.5295990295081565
1059400000 1.7730249372745375
1059500000 2.3323660246565505
1059600000 1.8007360487856119
1059700000 1.4878746085625112
1059800000 2.139221900725321
1059900000 1.8406090529867751
1060000000 1.6554830072917044
1060100000 1.8549230515097856
1060200000 2.1947197099688474
1060300000 1.3844474678786118
1060400000 1.8077272843515357
1060500000 2.199636284942642
1060600000 1.6704320482404171
1060700000 1.4147183978417728
1060800000 2.124288707600838
1060900000 1.7957355966945667
1061000000 1.4020081617797324
1061100000 2.048607556753117
1061200000 2.0154578014252547
1061300000 1.324564275289846
1061400000 1.6847225505418228
1061500000 2.034814910603213
1061600000 1.3745404469192424
1061700000 1.61515

1084900000 0.3597211457495273
1085000000 0.342056048771143
1085100000 0.3114220686397082
1085200000 0.30038847044720846
1085300000 0.32956302791871755
1085400000 0.3151436160075784
1085500000 0.28788808333420157
1085600000 0.3334377620621489
1085700000 0.34059777935452795
1085800000 0.17698572198429108
1085900000 0.09392947534834213
1086000000 0.07478865170697437
1086100000 0.10132545225255886
1086200000 0.10051460692291392
1086300000 0.1208605909736428
1086400000 0.33214665548185907
1086500000 0.2604433548018098
1086600000 0.24776843295647963
1086700000 0.287787139925015
1086800000 0.26687556180612976
1086900000 0.26163082733753523
1087000000 0.27502758127068877
1087100000 0.2594479993790216
1087200000 0.2862329041437745
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503
1087300000 0.24675951560076279
best so far: 0
type: [2, 1, 23, 1, 3] 138
cases of this type: 42849
1087400000 0.08155625710255372
best so far: 0
type: [2, 1, 23, 3, 1] 138
cases of this type: 14283
b